In [ ]:
#!/usr/bin/env python3
"""
API Loop Client for streaming responses - Clean Output Version
Sends POST requests to the AI search endpoint and shows only the streamed text
"""

import requests
import json
import sys
from datetime import datetime
from typing import Optional
import uuid
from IPython.display import display, Markdown, clear_output

class SimpleAPIClient:
    def __init__(self, base_url: str = "http://0.0.0.0:8000"):
        self.base_url = base_url
        self.endpoint = f"{base_url}/aisearch_fastmode/single_entities"
        self.headers = {
            'Accept': '*/*',
            'Accept-Encoding': 'gzip, deflate, br',
            'Connection': 'keep-alive',
            'Content-Type': 'application/json',
            'User-Agent': 'Python API Loop Client',
            'X-API-Key': 'Bearer dev_api_key_for_testing'
        }
        self.accumulated_text = ""
    
    def stream_response(self, response):
        """
        Stream and display text content as markdown in real-time
        """
        self.accumulated_text = ""
        
        try:
            for line in response.iter_lines(decode_unicode=True):
                if line.strip() and line.startswith("data: "):
                    data_content = line[6:]  # Remove "data: " prefix
                    
                    try:
                        json_data = json.loads(data_content)
                        
                        if "event" in json_data:
                            event = json_data["event"]
                            
                            # Extract and display text from text_line events
                            if event.get("step") == "text_line":
                                text = event.get("text", "")
                                if text:
                                    self.accumulated_text += text
                                    clear_output(wait=True)
                                    display(Markdown(self.accumulated_text))
                    
                    except json.JSONDecodeError:
                        continue
        
        except Exception:
            pass  # Silent error handling
    
    def send_request(self, query: str, conversation_id: str = None, prompt_id: int = 1, 
                    is_demographics_blocked: bool = False) -> bool:
        """
        Send a POST request and stream the response text
        """
        if conversation_id is None:
            conversation_id = f"test-{datetime.now().strftime('%Y%m%d-%H%M%S')}-{str(uuid.uuid4())[:8]}"
        
        payload = {
            "query": query,
            "conversation_id": conversation_id,
            "prompt_id": prompt_id,
            "isDemographicsBlocked": is_demographics_blocked
        }
        
        try:
            response = requests.post(
                self.endpoint,
                headers=self.headers,
                json=payload,
                stream=True,
                timeout=60
            )
            
            response.raise_for_status()
            
            # Stream and display the response as markdown
            self.stream_response(response)
            
            return True
            
        except Exception:
            print("Error: Failed to get response")
            return False

# Simple usage function
def run_aisearch(query: str, base_url: str = "http://0.0.0.0:8000"):
    """
    Simple function to run aisearch see the streamed response
    """
    client = SimpleAPIClient(base_url)
    return client.send_request(query)

print("Simple API Client loaded! Use run_aisearch('your query') to get started.")

In [42]:
query = "I need someone who worked at a company that made a big impact'"

In [44]:
run_aisearch(query)

Finding individuals who worked at a company that made a big impact.

True

In [20]:
run_aisearch(query)


1. I'm sorry, but I cannot assist with that request.

2. Can you give me the name of the company for which you are recruiting? This will help me find more targeted candidates.

3. Would you also like to target similar titles, such as **Vice President of Finance** or **Finance Vice President**?

4. Are there any specific industries, product areas, or types of companies the ideal candidate should come from?


True

In [21]:
run_aisearch(query)


1. Would you also like to target similar titles, such as **Vice President of Finance** or **President of Finance**?

2. I'm sorry, but I can only help with search queries for people, companies, or products. Could you please provide a search‑related request?

3. Are there any specific industries, product areas, or types of companies the ideal CFO should come from?


True

In [ ]:
run_aisearch(query)

In [ ]:
run_aisearch(query)

In [ ]:
run_aisearch(query)

In [ ]:
run_aisearch(query)

In [ ]:
run_aisearch(query)

In [ ]:
run_aisearch(".")

In [ ]:
run_aisearch(".")

In [ ]:
run_aisearch(".")

In [ ]:
run_aisearch(".")

In [ ]:
run_aisearch(".")

In [ ]:
run_aisearch("CFOs working in Google.")

In [ ]:
run_aisearch("I am looking to hire a CFO for Google from Dev Tools Companies.")

In [ ]:
run_aisearch("Big litigation partners who are generalists like Karen Dunn")

In [ ]:
run_aisearch("Big litigation partners who are generalists like Karen Dunn")

In [ ]:
run_aisearch("Big litigation partners who are generalists like Karen Dunn")

POST
/people/aisearch/expansion/filters_expansion
Ai Search Expansion


POST
/people/aisearch/expansion/industry_suggestion
Ai Search Suggestions


POST
/people/aisearch/extract_all_filters
Ai Search All Filters1


POST
/people/aisearch/context_modification
Context Modification


POST
/people/aisearch/updations
Ai Search Mapped Person


POST
/people/aisearch/variants
Ai Search Variants


POST
/people/aisearch/converse
Ai Search Fast




Of course. Here is a detailed and robust prompt designed to perform intent analysis and classify queries, encapsulated within a Python multi-line string.

This prompt is structured with a clear role, a primary goal, nuanced definitions, and specific rules for handling the conflicting conditions you identified, making it highly effective and less prone to errors.

```python
prompt = """
You are an expert AI assistant specializing in semantic analysis for the talent acquisition and executive search industry. You have a deep understanding of the nuances that differentiate a query intended for hiring from a query intended for market research.

Your primary task is to analyze a user's query, determine its underlying intent, and classify it as either a 'Recruiting Query' or a 'Search/Research Query'. You must base your decision on a deep analysis of the query's semantics, not just on superficial keywords.

---

## Core Definitions

1.  **Recruiting Query:**
    * **Primary Intent:** To find *people to hire* for a specific role, either explicitly stated or implied.
    * **Language:** The language is **transactional and action-oriented**. It often includes phrases like "I am looking for," "Find me candidates," "suitable for," or implies a need to fill a position.
    * **Focus:** The query is **person-centric**. The ultimate subject is an individual or a group of potential candidates.
    * **Contextual Clues:** It may contain criteria related to a candidate's career journey, such as "step-up" potential ($VP \rightarrow CFO$), or use company attributes (e.g., "PE-backed," "multibillion dollar") as a proxy for required *candidate experience*.

2.  **Search/Research Query:**
    * **Primary Intent:** To gather *information or data* about a market, a list of companies, or a general group of people for analysis, not for immediate hiring.
    * **Language:** The language is **informational and exploratory**. It often includes phrases like "List of," "What are," "search for," or simply states the entity of interest.
    * **Focus:** The query is **entity-centric**. The ultimate subject can be companies, market trends, or a broad demographic.
    * **Contextual Clues:** The filters used are purely descriptive of a market segment (e.g., "hospitals in Alaska," "companies in logistics"). There is no mention or implication of a candidate's career progression or fit for a specific, unstated client.

---

## Critical Rules for Handling Ambiguity

You must follow these rules to resolve conflicts and correctly classify difficult queries.

* **Rule 1: Job Titles Are Not a Decisive Signal.**
    * Both query types can contain job titles. Do NOT classify a query as 'Recruiting' just because it contains a title like "CFO" or "VP." You must find stronger, corroborating evidence of hiring intent.

* **Rule 2: Identify the Ultimate Subject.**
    * Analyze the query's syntax. Is the goal a list of *people* or a list of *companies/entities*?
    * *Example:* "Find me **CEOs** at PE-backed companies" is about a person (Recruiting). "List **PE-backed companies**" is about an entity (Search/Research).

* **Rule 3: Implicit Hiring Intent is Key (The Sourcing Rule).**
    * **This is the most important rule.** The first stage of recruiting is called "sourcing" or "market mapping," which often uses the language of research.
    * If a query seeks a list of *specific people* defined by their professional titles, skills, and company types, it has a **strong implicit hiring intent** because the goal is to build a candidate pipeline.
    * Therefore, you MUST classify these sourcing queries as **Recruiting Queries**, even if they use words like "list," "search for," or "look for."

---

## Output Format

You must provide your response in the following structured format. Do not add any conversational text outside of this structure.

**Classification:** `[Your classification: "Recruiting Query" or "Search/Research Query"]`
**Confidence:** `[A score from 0.0 to 1.0 indicating your confidence]`
**Reasoning:** `[A step-by-step explanation for your classification. Reference the Core Definitions and Critical Rules above to justify your decision.]`
**Key Signals:** `[A list of the specific words or phrases from the query that led you to your conclusion.]`

---

## Examples

**Example 1 Query:** "I am looking for Regional Finance Directors in Latin America with international experience and ideally in an industry related to fruit and vegetable production."

**Classification:** `Recruiting Query`
**Confidence:** `1.0`
**Reasoning:** `The query uses the transactional phrase "I am looking for," making the intent explicitly about finding a person. It specifies a role ("Regional Finance Directors"), a location, and required candidate experiences ("international experience," "fruit and vegetable production"). This aligns perfectly with the definition of a Recruiting Query.`
**Key Signals:** `["I am looking for", "Regional Finance Directors", "with international experience"]`

**Example 2 Query:** "PE-backed companies in logistics"

**Classification:** `Search/Research Query`
**Confidence:** `1.0`
**Reasoning:** `The query's ultimate subject is "companies," not people. It is a request for a list of entities based on descriptive market traits ("PE-backed," "in logistics"). It lacks any language indicating hiring intent. This aligns with the definition of a Search/Research Query.`
**Key Signals:** `["companies", "in logistics"]`

**Example 3 Query:** "Look for vice presidents of medical economics who currently or in the past worked for companies like Anthem, Aetna, or Elevance."

**Classification:** `Recruiting Query`
**Confidence:** `0.95`
**Reasoning:** `While the query uses the exploratory phrase "Look for," its ultimate subject is a list of specific people ("vice presidents of medical economics"). It specifies professional titles and target companies, which is a classic sourcing technique to build a candidate pipeline. This falls under Critical Rule #3 (The Sourcing Rule), as it demonstrates a strong implicit hiring intent.`
**Key Signals:** `["vice presidents of medical economics", "worked for companies like Anthem, Aetna"]`

---

Now, analyze the following user query:
"""
```

Factor
Recruiting Query
Search/Research Query
Role / Title
Always explicitly specifies a role or position (CEO, CFO, Director, VP, etc.).
May or may not mention roles; often general (e.g., “companies in aerospace” or “people in healthcare”).
Hiring Intent
Clearly framed as seeking to fill a role for a client. Uses phrasing like “I am looking for,” “Find me candidates,” or “Recruiting query.”
No hiring intent; focused on insights, lists, or mapping (e.g., “PE-backed companies in logistics”).
Seniority / Step-Up
Mentions whether step-up candidates (e.g., VP → CFO) are acceptable, and specifies acceptable company size for step-ups.
Not relevant; no mention of candidate progression.
Client / Search Context
May reference or request the client name, company size, AUM, or sub-sector to refine the candidate pool.
Purely descriptive, not tied to a specific client or placement.
Experience / Skills
Lists required candidate experiences (PE-backed, Sukuk, Lean Six Sigma, UHNW client management, etc.).
Lists company traits or market traits (e.g., “hospitals in Alaska”).
Outcome
Output is a candidate list matching hiring criteria.
Output is a market map, company list, or research data, not direct candidates.



In [25]:

test_data = [
    {
        "category": "1. Unrelated Queries",
        "scenario_description": "User asks the system to perform a task completely outside of its search and recruitment capabilities.",
        "conversation_history": [],
        "last_query": "Please write a polite rejection email for a candidate named John Doe.",
        "expected_output": {
            "ambiguity": 1,
            "follow_up": "I am designed to assist with searching for people, companies, and products for recruitment purposes. I cannot perform tasks like writing emails."
        },
        "reasoning": "The query asks for an action (writing an email) that is explicitly not supported by the AI Search System. This falls under the 'Ambiguous_Scenarios' rule for out-of-scope requests."
    },
    {
        "category": "1. Unrelated Queries",
        "scenario_description": "User asks a general knowledge question.",
        "conversation_history": [],
        "last_query": "What is the capital of Mongolia?",
        "expected_output": {
            "ambiguity": 1,
            "follow_up": "My capabilities are focused on search and recruitment. I am unable to answer general knowledge questions."
        },
        "reasoning": "This is an out-of-scope request. The system should identify that it cannot answer general knowledge questions and politely inform the user."
    },
    {
        "category": "2. Related but Ambiguous Queries",
        "scenario_description": "User asks for people from a company but provides a vague, non-specific timeline.",
        "conversation_history": [],
        "last_query": "Find me software engineers who worked at Twitter recently.",
        "expected_output": {
            "ambiguity": 1,
            "follow_up": "To help narrow down the search, could you please specify what timeframe you consider as 'recently' (e.g., past year, past 2 years)?"
        },
        "reasoning": "The term 'recently' is an objective ambiguity related to a timeline. It's not a subjective quality. The system needs to clarify this for effective filtering."
    },
    {
        "category": "2. Related but Ambiguous Queries",
        "scenario_description": "User provides a single, vague keyword that could refer to people, companies, or products.",
        "conversation_history": [],
        "last_query": "Cybersecurity",
        "expected_output": {
            "ambiguity": 1,
            "follow_up": "Are you looking for people with skills in Cybersecurity, companies in the Cybersecurity industry, or products related to Cybersecurity?"
        },
        "reasoning": "The user's intent is unclear. A single keyword requires clarification to determine if the search is for people, companies, or products to utilize the system's filters correctly."
    },
    {
        "category": "2. Related but Ambiguous Queries",
        "scenario_description": "User provides contradictory filters in their initial query.",
        "conversation_history": [],
        "last_query": "I'm looking for an entry-level software engineer with at least 10 years of total working years.",
        "expected_output": {
            "ambiguity": 1,
            "follow_up": "I noticed the request is for an 'entry-level' role but also requires 'at least 10 years' of experience. Could you please clarify the experience level you're looking for?"
        },
        "reasoning": "The query contains contradictory objective filters ('entry-level' and '10+ years'). The system must ask for clarification as per the 'Ambiguous_Scenarios' instructions."
    },
    {
        "category": "3. Related and Clear Queries",
        "scenario_description": "A standard, well-defined search for people with specific roles, skills, and location.",
        "conversation_history": [],
        "last_query": "Find me a Director of Product Management with experience in SaaS and based in Austin, Texas.",
        "expected_output": {
            "ambiguity": 0
        },
        "reasoning": "This is a clear and unambiguous query. All elements (job title, skill/industry, location) map directly to the system's filtering capabilities. No clarification is needed."
    },
    {
        "category": "3. Related and Clear Queries",
        "scenario_description": "User provides a full job description. The system should assume the intent is to find people.",
        "conversation_history": [],
        "last_query": "We are seeking a dynamic Head of Marketing to lead our marketing team. The ideal candidate will have over 8 years of experience in B2B tech marketing, a proven track record in demand generation, and experience managing a team of at least 5 people. An MBA is preferred.",
        "expected_output": {
            "ambiguity": 0
        },
        "reasoning": "As per 'Non_Ambiguous_Scenarios' Rule #2, when a user provides a job description, the intent is assumed to be a search for people matching those requirements. The query is not ambiguous."
    },
    {
        "category": "3. Related and Clear Queries",
        "scenario_description": "Testing the non-ambiguity of subjective or qualitative terms.",
        "conversation_history": [],
        "last_query": "Get me a highly-motivated, proactive, and innovative sales executive from a fast-paced company.",
        "expected_output": {
            "ambiguity": 0
        },
        "reasoning": "'Non_Ambiguous_Scenarios' Rule #3 explicitly states not to clarify subjective or qualitative terms. 'Highly-motivated', 'proactive', 'innovative', and 'fast-paced' should be passed through to the search system as-is."
    },
    {
        "category": "3. Related and Clear Queries",
        "scenario_description": "Testing the non-ambiguity of job title acronyms.",
        "conversation_history": [],
        "last_query": "Find me an SVP of Ops from Amazon or Microsoft.",
        "expected_output": {
            "ambiguity": 0
        },
        "reasoning": "'Non_Ambiguous_Scenarios' Rule #6 states that job title acronyms should not be clarified. The system should proceed with the search for 'SVP of Ops'."
    },
    {
        "category": "3. Related and Clear Queries",
        "scenario_description": "Testing the system's ability to handle queries for information about a specific, named entity.",
        "conversation_history": [],
        "last_query": "Show me the Financials for Oracle.",
        "expected_output": {
            "ambiguity": 0
        },
        "reasoning": "The query is about a specific, named company ('Oracle') and asks for a specific information modal ('Financials'). This is a clear, non-ambiguous request as per 'Non_Ambiguous_Scenarios' Rule #1."
    },
    {
        "category": "4. Ambiguous and Unrelated Queries",
        "scenario_description": "A query that is both vague and out of scope. The system should prioritize the out-of-scope nature.",
        "conversation_history": [],
        "last_query": "Help me organize my calendar.",
        "expected_output": {
            "ambiguity": 1,
            "follow_up": "I am designed to assist with searching for people, companies, and products. I cannot perform tasks like organizing calendars."
        },
        "reasoning": "While 'organize my calendar' is somewhat vague, the primary issue is that it's an out-of-scope task. The system should default to stating its purpose and limitations."
    },
    {
        "category": "5. Conversational Flow (Master_No_Repeat_Rule)",
        "scenario_description": "Tests the Master_No_Repeat_Rule with a vague user response. The system must not ask for clarification again.",
        "conversation_history": [
            {
                "role": "user",
                "content": "Find me developers from cool startups."
            },
            {
                "role": "assistant",
                "content": "{'ambiguity': 1, 'follow_up': 'Could you provide some examples of startups you consider 'cool', or specify an industry like FinTech, HealthTech, etc.?'}"
            }
        ],
        "last_query": "the tech ones",
        "expected_output": {
            "ambiguity": 0
        },
        "reasoning": "The user has responded to a clarification question. Although the response 'the tech ones' is still vague, the Master_No_Repeat_Rule overrides all other considerations. The system MUST set ambiguity to 0 and proceed without re-asking."
    },
    {
        "category": "5. Conversational Flow (Master_No_Repeat_Rule)",
        "scenario_description": "Tests the Master_No_Repeat_Rule with a minimal 'yes/no' response.",
        "conversation_history": [
            {
                "role": "user",
                "content": "I need a data person."
            },
            {
                "role": "assistant",
                "content": "{'ambiguity': 1, 'follow_up': 'By 'data person,' are you looking for a Data Scientist, Data Analyst, or Data Engineer?'}"
            }
        ],
        "last_query": "yes, proceed",
        "expected_output": {
            "ambiguity": 0
        },
        "reasoning": "The user's response directly addresses the assistant's previous question. Even though it doesn't specify which role, the Master_No_Repeat_Rule mandates that the conversation has been clarified and the system must not ask again."
    },
    {
        "category": "6. Modification Commands",
        "scenario_description": "User asks to remove a specific filter from a previous search. This should not be ambiguous.",
        "conversation_history": [
            {
                "role": "user",
                "content": "Find me product managers from Google with over 5 years of role tenure."
            }
        ],
        "last_query": "remove the role tenure filter",
        "expected_output": {
            "ambiguity": 0
        },
        "reasoning": "As per 'Non_Ambiguous_Scenarios' Rule #7, a command to remove a *specific* filter is not ambiguous. The system should proceed with the modified search."
    },
    {
        "category": "6. Modification Commands",
        "scenario_description": "User asks to remove all filters. This is the one modification command that is considered ambiguous.",
        "conversation_history": [
            {
                "role": "user",
                "content": "Find me product managers from Google with over 5 years of role tenure."
            }
        ],
        "last_query": "now remove all filters",
        "expected_output": {
            "ambiguity": 1,
            "follow_up": "Let me know if there is anything else you would like to add to the search?"
        },
        "reasoning": "The instruction under 'Ambiguous_Scenarios' specifies that only the command to 'remove **all** filters' is ambiguous and requires this exact follow-up question."
    },
    {
        "category": "7. Clever/Edge Cases",
        "scenario_description": "User asks for 'people like' a famous person, which is a supported feature.",
        "conversation_history": [],
        "last_query": "Find me people like Satya Nadella",
        "expected_output": {
            "ambiguity": 0
        },
        "reasoning": "The system information explicitly states that 'People like Satya Nadella' is a non-ambiguous query that can be handled. It falls under 'Non_Ambiguous_Scenarios'."
    },
    {
        "category": "7. Clever/Edge Cases",
        "scenario_description": "User asks for companies based on a complex, descriptive filter, which is a supported feature.",
        "conversation_history": [],
        "last_query": "Show me companies that used python before but are now not using python and have a revenue of over $1 Billion.",
        "expected_output": {
            "ambiguity": 0
        },
        "reasoning": "The 'Information' section about the system's capabilities explicitly lists this type of complex, descriptive query for companies as a feature. Therefore, it is not ambiguous ('Non_Ambiguous_Scenarios' Rule #5)."
    },
    {
        "category": "7. Clever/Edge Cases",
        "scenario_description": "User asks the system to explain a previous question.",
        "conversation_history": [
            {
                "role": "user",
                "content": "Find me developers from new ventures."
            },
            {
                "role": "assistant",
                "content": "{'ambiguity': 1, 'follow_up': 'Could you clarify if by 'new ventures' you mean companies that are VC Funded, recently founded, or something else?'}"
            }
        ],
        "last_query": "what do you mean by vc funded?",
        "expected_output": {
            "ambiguity": 1,
            "follow_up": "I was asking if you'd like to search for companies that have received funding from venture capital firms. Would you like to apply this filter?"
        },
        "reasoning": "This falls under the ambiguous scenario where the 'user asks for explanation'. The system should rephrase its previous question in simpler terms to clarify the user's options."
    }
]


In [ ]:
for item in test_data:
    query = item["last_query"]
    

In [ ]:
MODIFICATION_DETECTION_PROMPT = """
<Role>
Your primary task is to analyze the complete `<Whole_Conversation>` between a user and an AI system. This conversation includes user queries, system follow-ups, and various results across different filters. Your main goal is to analyze the `<Last_Query>` as a continuation of the conversation, and analyze the conversation and follow ups and results to detect the actual intented and implied modifications of the user across different filters and then generate a modification query that contains all the necessary modifications that need to be made across appropriate filters.
</Role>


<System_Information_and_Attributes>
We have an AI search system which basically takes the `modification_prompt` , `attributes` and `indexes` and then use this information to perform modifications and retrieve the results according to the user needs. 

Our System has the following Attributes:
<attributes>
    • `job_role`: Handles job titles, management levels and the number of employees at the company where that title was held.
    • `company_industry_product`: Handles Industries, Companies, Company Location, organizations, or products.
    • `skill`: Specific skills or key terms.  
    • `location`: Handles person location.
    • `total_working_years`: Handles tenure in a company, tenure in a role and total career experience.
    • `education`: Specific schools, degrees, or certifications.  
    • `name`: Exact person’s name.  
    • `ownership`: Include the ownership type—selecting from "Public," "Private," "VC Funded," or "Private Equity Backed" (where "Private Equity Backed" refers to an investee company, not an investor company)—only when it is explicitly mentioned in the current context. Do not infer ownership types from queries like "get people from startups" or "get people from Fortune 500," as these do not explicitly request ownership information.
    • "demographics": Handles gender, ethnicity and age.
</attributes>

<indices>
The `indices` are the result indices from the previous AI search results that are affected by the modifications. The `indices` key will contain the relevant result index, which will be taken from the <Whole_Conversation> inside <Result-[Index]> tags.
</indices>
</System_Information_and_Attributes>


<System_Information_and_Details>
Here is an information section on the kind of information you will mostly be dealing with. This is the context for the search system you are creating a modification prompt for.
    ### Core Search
    * **Entities**: People, Companies, Products.
    * **Identifiers**: Can be found by name or LinkedIn URL/ID.

    ### Person Search Filters
    * **`name`**: Individual's name.
    * **`skill/keywords/industry`**: Must match at least one specified skill or industry.
    * **`company`**: Current or past employers.
    * **`location`**: Geographic area.
    * **`school`**: Educational institution.
    * **`education`**: `{"degree": "...", "major": "..."}`
        * **Degrees**: `Associate, Bachelor's, Master's, Doctorate, Diploma, Certificate` etc
    * **Tenure**: `company_tenure` (current company), `role_tenure` (current role), 
    * **Years of Experience**: `total_working_years` (career range).
{{include_gender_age_ethnicity}}

    ### Company & Product Search Parameters
    * **`ownership`**:  Some examples of company ownership are: "Public", "Private", "VC Funded", "Private Equity Backed", "VC backed" etc.
    * **Descriptive Search**: Find companies via complex descriptions (e.g., financial growth, tech stack changes, "Find CEOs from IPO Ready Companies").
    * **Product Search**: Find products/services by name (e.g., "Wearables").

    ### Available Data
    * **For People**: Summary, Experience, Education, Information, Pay Progression, Similar Profiles.
    * **For Companies**: Summary, Financials, M&A, Competitors, Reports, Business Units, News, Products.

</System_Information_and_Details>

<Intent_Analysis>
- You must first perform a very careful intent analysis of the <Last_Query> in the context of the <Whole_Conversation>, follow_ups, user_queries, results, etc., and carefully identify what is it that the user wants.
- <Last_Query> is the continuation of the <Whole_Conversation>, so any request in the <Last_Query> should always be analyzed as the continuation of the conversation. It must not be considered a new request unless explicitly told by the user.
- If he excludes anything, then carefully identify, whether user does not want a specific entity, or the intent is to exclude a specific category. For Example: "Crowd is not a pureplay company, remove it". Here the user's actual intention is that he wants pureplay companies with the provided description, so the intent is actually to **only keep pureplay companies and remove non pureplay companies**.
- Therefore, you must carefully analyze the intent of the user.
</Intent_Analysis>

<Modification_Analysis>
- After carefully identifying the correct intent of the user, you need to identify the necessary and required modifications to the filters in the result.
- After that you need to identify whether those modifications have effects on other filters or not, and make the implied modifications in them as well, i.e.,  If the user adds or remove anything, then also you need to identify the implication of that to other filters, the two closely related filters for this are location and companies. If there is a geographical change and that new targeted geography is not appropriately represented in the company prompts, then you need to make it change the company filters as well.
- After carefully identifying the necessary modification, you must identify the list of the attributes that require modification. 
- Attributes to modify MUST only contain from the following list: ["skill", "location", "job_role", "company_industry_product", "total_working_years", "education", "name", "ownership", "demographics"]. These are the names of the agents which handle their specific filters so attributes must only contain one of these and NONE OTHER. 
- Identify the Result-Indices to which the modification should be applied, (the last index's results MUST CONTAIN the "AI_Search_Results" which require modification)
</Modification_Analysis>




<Very_Important_Guidelines_For_Writing_Modification_Prompt>
# Core Principles:
    - After correctly identifying all the required modifications write a `modification_prompt` will be treated as an instruction describing how the previous filters should be modified (e.g., "Change location to USA", "Remove skills", "Add age 50+ and location Europe").
    - **The modification_prompt should be a direct and straight-forward instruction**: "Add this", "Change this", etc.**
    - If can **NEVER** detect ambiguity, or **ASK ANY QUESTIONS**.
    - If unsure, You MUST assume the most **plausible and relevant meanings** according to the context, and must always write a modification_prompt.
    - **Don't miss any details: Don't shorten or summarize important details, such as certain descriptions of companies or other requirements**

# Guidelines:
    - Review everything: Look at the <Last_Query> in light of the entire conversation.
    - Include all details: Don't miss any information or requirements.
    - Be clear: Use straightforward language.
    - Keep specifics: Don't shorten or summarize important details.
    - If some details for specific companies are given, you must include them in the modification_prompt as well.
    - If anything in the <Last_Query> is ambiguous, you must assume the most **plausible and relevant meanings** and write the modification_prompt accordingly.
</Very_Important_Guidelines_For_Writing_Modification_Prompt>


## Output Format: Your output format must be in this exact format below:
<llm_response>
    <reasoning>
    [Write your entire though process and reasoning here]
    </reasoning>

    Return a JSON object enclosed in <Output> </Output> tags.
    <Output>
    {
            "modification_prompt": "",
            "attributes": [""], # Can only contain values from the list ["skill", "location", "job_role", "company_industry_product", "total_working_years", "education", "name", "ownership", "demographics"]. 
            "indexes": [0,1], # last index must contain the AI_SEARCH_RESULTS to modify
    }
    </Output>





</llm_response>

"""

"""
Based on the instructions provided for different scenarios, you must first identify which of these scenarios apply to the conversation. Then, your main goal is to generate a **single, comprehensive prompt**, that functions as a detailed search query that contains the entire context and detailed information of what the user wants according to the guidelines without missing anything important. It will be provided to a search system (details of which are provided below) to retrieve the most relevant results, so make sure to pick all the necessary details according to the guidelines and instructions mentioned below.
</Role>
"""